# CCTV Cross-Camera Person Tracking and Analysis
# ============================================

This notebook implements cross-camera person tracking and re-identification using:

- YOLOX for person detection
- ByteTracker for single-camera tracking
- Deep Person ReID for cross-camera person re-identification
- InsightFace for demographic analysis

## Setup and Imports

In [1]:
import os
import sys
from pathlib import Path
import logging
from datetime import datetime
import numpy as np
import cv2
import torch
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torchvision.transforms import transforms
import wget

# Deep Person ReID imports
import torchreid
from torchreid import models
from torchreid import utils

# Demographic analysis (using InsightFace)
from insightface.app import FaceAnalysis

c:\Users\mc1159\AppData\Local\anaconda3\envs\cctv-analysis\lib\site-packages\torchreid\reid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
c:\Users\mc1159\AppData\Local\anaconda3\envs\cctv-analysis\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## Configuration

In [ ]:
# YOLOX weights
import torchreid

model_folder_path = os.path.join("..", "models")


# Create model folder if it doesn't exist

Path(model_folder_path).mkdir(parents=True, exist_ok=True)

os.chdir(model_folder_path)


# Download YOLOX weights (choose one based on your needs)

wget.download(

    "https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth")


wget.download(

    "https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_m.pth")


wget.download(

    "https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_l.pth")


wget.download(

    "https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_x.pth")


# Then download the weights using torchreid's model zoo

# OSNet weights

torchreid.utils.download_url(

    'https://drive.google.com/uc?id=1vduhq5DpN2q1g4fYEZfPI17MJeh9qyrA',

    'osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'

)

NameError: name 'os' is not defined

In [2]:
class Config:
    # YOLOX Config
    YOLOX_EXP_FILE = "yolox_l"  # or yolox_m, yolox_l, yolox_x
    YOLOX_WEIGHTS = "yolox_l.pth"
    CONFIDENCE_THRESHOLD = 0.5

    # ByteTracker Config
    TRACK_BUFFER = 30
    TRACK_THRESH = 0.5

    # ReID Config
    REID_MODEL = 'osnet_x1_0'
    REID_WEIGHTS = 'osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth'

    # Video Processing
    BATCH_SIZE = 4
    INPUT_SIZE = (608, 1088)  # (height, width)

    # Matching Config
    MAX_COSINE_DISTANCE = 0.3
    # maximum time difference (in seconds) for matching
    MAX_TIME_DIFFERENCE = 300

In [3]:
# Change the working directory to the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(parent_dir)
# Check current working directory
print(os.getcwd())

c:\GitHub\cctv-analysis


## Analyse the CCTV footage